<a href="https://colab.research.google.com/github/Bhagwat-03/LiverNet/blob/main/initial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow numpy pandas scikit-learn matplotlib opencv-python


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

def load_data(image_dir, mask_dir):
    images, masks = [], []
    for img_name in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, img_name))
        img = cv2.resize(img, (256, 256)) / 255.0
        mask = cv2.imread(os.path.join(mask_dir, img_name), 0)
        mask = cv2.resize(mask, (256, 256)) / 255.0
        images.append(img)
        masks.append(mask[..., np.newaxis])
    return np.array(images), np.array(masks)

image_dir = "path/to/images"
mask_dir = "path/to/masks"
X, y = load_data(image_dir, mask_dir)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def unet_model(input_size=(256, 256, 3)):
    inputs = tf.keras.layers.Input(input_size)
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)

    up1 = tf.keras.layers.UpSampling2D((2, 2))(pool2)
    concat1 = tf.keras.layers.Concatenate()([up1, conv1])
    conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat1)
    output = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv3)

    return tf.keras.Model(inputs, output)

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    callbacks=[early_stopping, checkpoint])


In [ ]:
model.load_weights("best_model.h5")
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Predict on new images
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256, 256)) / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)[0]
    cv2.imshow("Prediction", pred)
    cv2.waitKey(0)

predict_image("i.jpg")
